In [8]:
import matplotlib 
matplotlib.use ("TkAgg") #TkAgg is a backend for matplotlib to render plots 

#we are going to make our own preprocess function 

import pandas as pd 
import seaborn as sns 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve 
from keras.models import Sequential
from keras.layers import Dense 
import matplotlib.pyplot as plt 
import numpy as np 
np.random.seed(16)


In [9]:
df = pd.read_csv('C:\Real junkies\sem 8\Datasets\diabetes.csv')

In [10]:
def preprocess_data(df):
    #replacing zero with NaN
    zero_columns = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]
    df[zero_columns] = df [zero_columns].replace(0, np.nan)

    #Then fill the missing values with teh column mean 
    df.fillna (df.mean(), inplace = True)

    #Feature scaling 
    scaler = StandardScaler()
    feature_columns = df.columns[df.columns != "Outcome"]
    df[feature_columns] = scaler.fit_transform(df[feature_columns])

    return df
    

In [12]:
df = preprocess_data(df) #Applying the preprocessing

In [13]:
X = df.loc [:, df.columns != 'Outcome']
y = df.loc [:, 'Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [15]:
model = Sequential()
model.add(Dense(32, activation = 'relu', input_dim = 8))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit (X_train, y_train, epochs = 200, verbose = False)

c:\Users\nirbh\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
#Results accuracy

scores = model.evaluate (X_train, y_train, verbose = False)
print ("Training Accuracy: %.2f%%\n" % (scores[1]*100))
scores = model.evaluate (X_test, y_test, verbose = False)
print("Testing Accuracy: %.2f%%\n" % (scores[1]*100))

Training Accuracy: 88.93%

Testing Accuracy: 70.78%



In [18]:
# Confusion matrix results 
y_test_pred = (model.predict(X_test) > 0.5).astype("int32")
c_matrix = confusion_matrix (y_test, y_test_pred)
ax = sns.heatmap (c_matrix, annot = True, xticklabels = ['No Diabetes', 'Diabetes'], yticklabels = ['No Diabetes', 'Diabetes'], cbar = False, cmap = 'Blues')
ax.set_xlabel ("Prediction")
ax.set_ylabel("Actual")
plt.show()
plt.clf()

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


In [19]:
y_test_pred_probs = model.predict (X_test)
FPR, TPR, _ = roc_curve (y_test, y_test_pred_probs)
plt.plot(FPR, TPR)
plt.plot([0,1],[0,1],'--',color = 'black')
plt.title('ROC Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()
plt.clf()

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
